# 爬蟲實戰 - 股票篇
學會爬蟲就可以加入網軍了嗎? 可以領1450嗎? 既然大家都被標題騙來了，就看下去吧ヾ(●゜▽゜●)

## Yahoo 股票及時報價

教學篇我們提到網站框架越新、設計越潮，爬蟲的難度就會越高。先來看看一些金融網站的設計吧~
1. Yahoo台灣 https://tw.stock.yahoo.com/
2. Yahoo金融 https://finance.yahoo.com/quote/TW/
3. 台灣證券交易所 https://www.twse.com.tw/zh/
4. Trading Economics https://tradingeconomics.com

這幾個網站對台積電(股票代號2330)的即時股價的頁面長這樣:
1. Yahoo台灣 https://tw.stock.yahoo.com/q/q?s=2330 
2. Yahoo金融 https://finance.yahoo.com/quote/2330.TW?p=2330.TW&.tsrc=fin-srch (可爬 需其他套件)
3. 台灣證券交易所 https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY.html (為動態查詢內容 同上)
4. Trading Economics https://tradingeconomics.com/2330:tt (同上)

[](pandas.read_html)
[](https://www.twse.com.tw/exchangeReport/STOCK_DAY_AVG?response=html&date=20200220&stockNo=2330)
[](https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20200220&stockNo=2330&_=1582185467917)

## 目標: 擷取今日的開、高、低、收&昨收
- 今天我們的對象是台積電，目標數字所在的欄位如下圖
![](https://drive.google.com/uc?export=download&id=1oEKgI1DGTx9nfJxW9GpfvUalSgfFpypl)


## 開始實作
不管是多複雜的網頁爬蟲，我們一樣採取2步驟戰略:「先建立連線，再解析網頁」

首先，我們先知道台灣yahoo網站的台積電網址，才能發出http請求已建立連線。

In [31]:
import requests

url = "https://tw.stock.yahoo.com/q/q?s=2330"
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}

res = requests.get(url, header)
res

<Response [200]>

第二步驟是透過BeautifulSoup套件來解析HTML網頁。

In [44]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, "html.parser")

其實解析完HTML網頁，你會發現好像沒甚麼頭緒，因為parse出來的標籤(tags)太多太亂了，還有混雜CSS、JavaScript在裡面(๑•́ ₃ •̀๑)

但是我們知道目標資料都在一個表格裡面，表格在HTML格式中代表的標籤是`<table>`，先按下F12(或是滑鼠右鍵>檢查)嘗試找出`<table>`這個標籤在哪裡吧。順帶一提，表格`<table>`中的每一列是一個`<tr>`(table row)，每列中會有許多資料`<tr>`(table data)。

```html
<table>
    <tr>
        <td>欄位名稱</td>
        <td>欄位名稱</td>
        ...
    </tr>
    <tr>
        <td>data</td>
        <td>data</td>
        ...
    </tr>
</table>
```

In [35]:
import pandas as pd
dfs = pd.read_html(res.text)
dfs

[    0                1
 0 NaN  資料日期: 109/02/20,                                                    0
 0  股票代號  時間  成交  買進  賣出  漲跌  張數  昨收  開盤  最高  最低  ...,             股票代號     時間     成交     買進     賣出    漲跌     張數     昨收     開盤  \
 0  2330台積電加到投資組合  14:30  325.5  325.5  326.0  ▽1.0  26923  326.5  328.0   
 
       最高     最低                個股資料  
 0  329.0  325.0  成交明細技術　新聞基本　籌碼個股健診  ,         0   1
 0  相關贊助網站 NaN
 1  相關贊助網站 NaN,         0   1
 0  相關贊助網站 NaN,                                                    0
 0  Yahoo Finance 服務條款 隱私權 台股資料來源臺灣證券交易所、臺灣期貨交易所及財...]

In [40]:
dfs[2]

,股票代號,時間,成交,買進,賣出,漲跌,張數,昨收,開盤,最高,最低,個股資料
0,2330台積電加到投資組合,14:30,325.5,325.5,326.0,▽1.0,26923,326.5,328.0,329.0,325.0,成交明細技術 新聞基本 籌碼個股健診
